### Initial Comment

This script allows to access the Auchan website and retrieve the data that is stored on the website specific tags.

Using the requests and the bs4 libraries was accessed the html code of the different webpages and retrieved the information regarding the name and the price of the products that were listed there.

The data that was obtained via this method was then transformed into a string and manipulated to get the product name and price on a readable format.

Based on the excel file _Categories.xlsx_, that was inputed, it was defined that the code would loop through the names of the products and would search for that name on the products webpage of the Auchan website.
 - the group opted to create a new url based on the "base url" of the webpage and replacing the words that had the search criteria for the ones on this file.

When accesing a url the data was stored on a data frame. This df will then be filtered using the function data_cleaning, which removes all searches that are out of scope for that search.
 - for example, when searching meat products is usual to have animal food on the search results, which is out of the scope of the search, therefore these items would be removed
Based on the removed results then the products would be narrowed for the top 10 products, that were selected based on the search preferences that each supermarket marked as more relevant.

These individual product dataframes were then concatenated into one dataframe that stores the information of this specific supermarket.

To this dataframe are then created new columns that have some summary statistics of the data on each row, detailing the minimum price, maximum and average price for each row, as well as the number of products that were passed onto the final dataframe for each product.

These data are to be then used on the next steps of the project


In [ ]:
import requests
from bs4 import BeautifulSoup
import re # For string transformation
import pandas as pd
import unidecode

In [2]:
# Read the initial excel file with all product names to be searched on and all the string expections
data = pd.read_excel("Categories.xlsx")

data["Product"]= data["Product"].replace('Vaca', 'Bovino', regex=True)
# For this supermarket the term "Vaca" is not used to search for the meat products, therefore a replacement for 'Bovino' must be done

data_subset = data.iloc[:,:1]

## String transformation to remove all uppercase and all accents (text normalisation)
data_subset = data_subset.applymap(lambda row: unidecode.unidecode(row).lower() if type(row) == str else row)


data_subset.head()

,Product
0,peru
1,frango
2,bacalhau
3,cebola
4,batata


### Explanation Comment
The below function retrieves the data based of the Continente's product's web-page.

Based on the information on the web-page it will create a dataframe with all the Product Names and respective Product Prices on that page.

In [3]:
def get_name_price(url):
    
    #1. Get web data into the notebook
    page = requests.get(url)
    if page.status_code == 200:
        pass
    elif page.status_code == 404:
        print('This page was not found.')
        return 
    
    soup = BeautifulSoup(page.text, 'html.parser')
    
    
    number_products_price = len(soup.find_all('span', class_ = 'auc-measures--price-per-unit'))
                          
    product_prices = soup.find_all('span', class_ = 'auc-measures--price-per-unit') # get price based on Auchan's website design
    product_names = soup.find_all('a', class_ = 'link') # get name based on Auchan's website design
    
    number_products_price = len(product_prices)
    number_products_name = len(product_names)
    
    if number_products_price != number_products_name:
        # The number of product names is different from the number of product prices
        print("The number of prices and names on this webpage is different, please check this page manually. \nNot all products were retrieved")
        #return
        
    product_name_list = []
    product_price_list = []
    
    for i in range(0, min(number_products_price, number_products_name)):
        product_name = str(product_names[i])
        product_price = str(product_prices[i])
        
    
        # 2. String formatting
        # 2.1. Transformation of Product Name:
        product_name = re.sub('<[^>]+>', '', product_name)
        
        
        # 2.2. Transformation of Product Price
        product_price = str(product_prices[i])
        product_price = re.sub('<[^>]+>', '', product_price)
        product_price = re.sub('\n', '', product_price)
        product_price = re.sub('€/un', '', product_price) # Removal of €/un
        product_price = re.sub('€/Kg', '', product_price) # Removal of Kg
        product_price = re.sub('€/Lt', '', product_price) # Removal of Lt
        product_price = re.sub('€/L', '', product_price) # Removal of Lt
        product_price = re.sub(' ', '', product_price)
        product_price = re.sub(',', '.', product_price)
        product_price = re.sub('Pricereducedfrom', '', product_price)
        product_price = re.sub('to', '', product_price)
        
        
        # 3. Append product name and price to the correspondent lists
        product_name_list.append(product_name)
        product_price_list.append(product_price)
        
    # 4. Create a dataframe to store all the data
    output_df = pd.DataFrame(list(zip(product_name_list,product_price_list)), columns = ["Product Name", "Product Price"])
    return output_df

### Explanation Comment
The below **_function_** allows the definition of the data cleaning process based on the "forbiden strings" for each product type
 - Please be aware that the column position on the original dataframe that has the information regarding the forbiden strings is hard coded and at the moment has the column number = 2. If the original data set changes then we need to adapt this position

This function sets up the top 10 products for each search object, and removes from it some user defined strings that may confound the products to be analysed.
 - i.e. if we search for "peru" we may have some results regarding animal food that has turkey, and also some other meat products made from turkey. These can be defined initially to avoid including these data on the final dataframe

In [4]:
def data_cleaning(url_data, index, original_data = data):
    
    strings_to_remove = original_data.iloc[index, 2] # get the set of strings to remove
    
    if pd.isna(strings_to_remove) == True:
    # Meaning that the forbiden strings field is empty
        strings_to_remove = ""
    
    data_10 = url_data[url_data["Product Name"].str.contains(strings_to_remove,)==False] # Include the specific strings that may confound the search for this product
    
    data_10 = data_10.iloc[:10,:] # Search for the top 10 products sorted by relevance and from the main brand7

    return data_10

### Explanation Comment
The below loop will iterate across all products that we've defined in the initial excel.

The **search_url** object has a fixed structure that will define the website url to be search on.
 - Using this url the function **get_name_price** will be called to retrieve the data from the web using the built url
 - The data will then be cleaned to have the top10 products on each search term that fits the description

The output of each iteration will be a dataframe with all relevant product names and prices for each product that is being searched

In [5]:
# For tests, when deploying please remove the if condition so that it can run for all product that are placed on the initial excel


for index, row in data_subset.iterrows():
    if index == 0:
        search_object = row["Product"]
        search_url = str("https://www.auchan.pt/pt/pesquisa?q=" + str(search_object) + "+auchan&search-button=&lang=pt_PT")
        strings_to_remove = data.iloc[index, 2] # get the set of strings to remove
  
        ## Call the function to retrieve the data from the web
        all_data = get_name_price(search_url)
        
        all_data["Supermarket"] = "Auchan"
        all_data["Category Object"] = data.iloc[index, 1] # Adding Category to the dataframe
        all_data["Product Object"] = data.iloc[index, 0] # Adding Product to the dataframe
        
        # Clean the results obtained so that we can have only the results that are of interest to us
        product_data = data_cleaning(url_data = all_data, index = index, original_data = data)
        aggregate_data = product_data # pass the data to the aggregate data frame 
        
    else:
        
        search_object = row["Product"]
        search_url = str("https://www.auchan.pt/pt/pesquisa?q=" + str(search_object) + "+auchan&search-button=&lang=pt_PT")
        strings_to_remove = data.iloc[index, 2] # get the set of strings to remove
  
        ## Call the function to retrieve the data from the web
        all_data = get_name_price(search_url)
        
        all_data["Supermarket"] = "Auchan"
        all_data["Category Object"] = data.iloc[index, 1] # Adding Category to the dataframe
        all_data["Product Object"] = data.iloc[index, 0] # Adding Product to the dataframe
        
        
        # Clean the results obtained so that we can have only the results that are of interest to us
        product_data = data_cleaning(url_data = all_data, index = index, original_data = data)
        
        aggregate_data = pd.concat([aggregate_data, product_data])


### Comment
Further transformations regarding how to aggregate the prices, and the number of products to be used should be discussed so that we all get the same uniformised version of the data frame

In [6]:
aggregate_data["Product Price"]=aggregate_data["Product Price"].replace(',', '.', regex=True)
aggregate_data["Product Price"] = aggregate_data["Product Price"].astype(float)

In [7]:
print(aggregate_data.dtypes)

Product Name        object
Product Price      float64
Supermarket         object
Category Object     object
Product Object      object
dtype: object


In [8]:
final_df = aggregate_data.groupby(['Supermarket','Category Object','Product Object']).agg({'Product Price': [("Average Per Product", "mean"),("Min Price per Product", "min"),("Max Price per Product", "max") ,("Count of Products", "count")]})
final_df

Product Price  \
                                             Average Per Product   
Supermarket Category Object Product Object                         
Auchan      Bebidas         Agua Garrafão               0.260000   
            Casa            Detergente Loiça            1.585000   
                            Detergente WC               1.980000   
                            Detergente chao             1.265714   
                            Papel higienico             0.386000   
                            Sacos do Lixo               0.084000   
            Charcutaria     Fiambre                     9.717000   
            Congelados      Douradinhos                 5.530000   
                            Ervilhas                    1.672000   
            Enlatados       Feijao                      1.413000   
                            Grão                        1.680000   
            Fruta           Maçã                        2.090000   
                            Pera                        2.390000   
            Higiene         Escova de Dentes            0.697000   
                            Pasta de Dentes             9.156000   
                            Sabonete                   11.041250   
                            desodorisante              30.097000   
            Laticinios      Iogurtes                    1.764000   
                            Leite                       0.831000   
                            Manteiga                    6.360000   
                            Ovos                        0.614000   
                            Queijo                      6.898000   
            Legumes         Abóbora                     3.035000   
                            Alho                        5.180000   
                            Alho Frances                1.190000   
                            Batata                      1.156667   
                            Bróculos                    1.495000   
                            Cebola                      1.525000   
                            Cenoura                     2.638333   
                            Couve Coração               1.190000   
            Mercearia       Arroz                       2.534000   
                            Azeite                      4.250000   
                            Açúcar                      1.765000   
                            Cafe                       13.236667   
                            Cereais                     4.129000   
                            Esparguete                  2.791667   
                            Farinha                     1.731000   
                            Oleo Alimentar              2.750000   
                            Vinagre                     2.487500   
            Padaria         Pão                         2.883750   
            Peixaria        Bacalhau                   11.249000   
                            Salmao                     18.488333   
            Talho           Bovino                     10.217000   
                            Frango                      4.760000   
                            Perú                        8.078000   
                            Porco                       5.230000   

                                                                    \
                                             Min Price per Product   
Supermarket Category Object Product Object                           
Auchan      Bebidas         Agua Garrafão                     0.20   
            Casa            Detergente Loiça                  0.05   
                            Detergente WC                     1.98   
                            Detergente chao                   1.26   
                            Papel higienico                   0.27   
                            Sacos do Lixo                     0.04   
            Charcutaria     Fiambre                           5.95   
            Congel

In [9]:
final_df.to_excel("Auchan_Products.xlsx")

In [10]:
final_df.index
final_df.loc["Auchan",:]

Product Price                        \
                                 Average Per Product Min Price per Product   
Category Object Product Object                                               
Bebidas         Agua Garrafão               0.260000                  0.20   
Casa            Detergente Loiça            1.585000                  0.05   
                Detergente WC               1.980000                  1.98   
                Detergente chao             1.265714                  1.26   
                Papel higienico             0.386000                  0.27   
                Sacos do Lixo               0.084000                  0.04   
Charcutaria     Fiambre                     9.717000                  5.95   
Congelados      Douradinhos                 5.530000                  5.53   
                Ervilhas                    1.672000                  1.09   
Enlatados       Feijao                      1.413000                  1.15   
                Grão                        1.680000                  1.21   
Fruta           Maçã                        2.090000                  1.99   
                Pera                        2.390000                  1.39   
Higiene         Escova de Dentes            0.697000                  0.33   
                Pasta de Dentes             9.156000                  2.15   
                Sabonete                   11.041250                  9.56   
                desodorisante              30.097000                  7.97   
Laticinios      Iogurtes                    1.764000                  1.09   
                Leite                       0.831000                  0.56   
                Manteiga                    6.360000                  5.96   
                Ovos                        0.614000                  0.10   
                Queijo                      6.898000                  5.10   
Legumes         Abóbora                     3.035000                  1.69   
                Alho                        5.180000                  5.18   
                Alho Frances                1.190000                  1.19   
                Batata                      1.156667                  0.99   
                Bróculos                    1.495000                  1.49   
                Cebola                      1.525000                  0.99   
                Cenoura                     2.638333                  1.09   
                Couve Coração               1.190000                  1.19   
Mercearia       Arroz                       2.534000                  0.95   
                Azeite                      4.250000                  4.25   
                Açúcar                      1.765000                  0.79   
                Cafe                       13.236667                  5.96   
                Cereais                     4.129000                  2.98   
                Esparguete                  2.791667                  1.29   
                Farinha                     1.731000                  0.89   
                Oleo Alimentar              2.750000                  2.75   
                Vinagre                     2.487500                  0.59   
Padaria         Pão                         2.883750                  1.42   
Peixaria        Bacalhau                   11.249000                  7.49   
                Salmao                     18.488333                  7.99   
Talho           Bovino                     10.217000                  5.99   
                Frango                      4.760000                  2.39   
                Perú                        8.078000                  3.75   
                Porco                       5.230000                  3.99   

                                                                          
                                 Max Price per Product Count of Products  
Category Object Product Object                                    